## pitch2017前処理

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
pd.set_option('display.max_Columns', 100)

In [2]:
train_pitch = pd.read_feather('data/train_pitch.f')
print(train_pitch.shape)

(257117, 51)


### 投手
#### 球種
0:ストレート 1:カーブ 2:スライダー 3:シュート 4:フォーク 5:チェンジアップ 6:シンカー 7:カットボール

In [3]:
train_pitch.replace({'球種': {
    0: '0_straight', 
    1: '1_curve', 
    2: '2_slider', 
    3: '3_shoot', 
    4: '4_fork', 
    5: '5_changeup', 
    6: '6_sinker', 
    7: '7_cutball'
}}, inplace=True)
train_pitch.rename(columns={'球種': 'ball'}, inplace=True)

In [4]:
pitch_ball = train_pitch[['投手ID','ball']]
pitch_ball = pd.get_dummies(pitch_ball, columns=['ball'])

In [5]:
groupby_id = pitch_ball.groupby('投手ID').sum()

In [6]:
groupby_id['curve'] =  groupby_id['ball_1_curve'] / groupby_id['ball_0_straight']
groupby_id['slider'] =  groupby_id['ball_2_slider'] / groupby_id['ball_0_straight']
groupby_id['shoot'] =  groupby_id['ball_3_shoot'] / groupby_id['ball_0_straight']
groupby_id['fork'] =  groupby_id['ball_4_fork'] / groupby_id['ball_0_straight']
groupby_id['changeup'] =  groupby_id['ball_5_changeup'] / groupby_id['ball_0_straight']
groupby_id['sinker'] =  groupby_id['ball_6_sinker'] / groupby_id['ball_0_straight']
groupby_id['cutball'] =  groupby_id['ball_7_cutball'] / groupby_id['ball_0_straight']
groupby_id['total'] = (groupby_id['ball_0_straight'] + groupby_id['ball_1_curve'] + groupby_id['ball_2_slider']
                       + groupby_id['ball_3_shoot'] + groupby_id['ball_4_fork'] + groupby_id['ball_5_changeup']
                       + groupby_id['ball_6_sinker'] + groupby_id['ball_7_cutball'])

In [7]:
pitch_ball = groupby_id.reset_index(inplace=False)

In [8]:
pitch_ball.drop(
    columns=[
        'ball_0_straight', 'ball_1_curve', 'ball_2_slider', 'ball_3_shoot', 
        'ball_4_fork', 'ball_5_changeup', 'ball_6_sinker', 'ball_7_cutball'
    ], inplace=True)

In [9]:
pitch_ball.head()

,投手ID,curve,slider,shoot,fork,changeup,sinker,cutball,total
0,11606,0.495751,0.000000,0.000000,0.407932,0.000000,0.000000,0.101983,708.0
1,11766,0.000000,0.967290,0.509346,0.000000,0.051402,0.149533,0.000000,573.0
2,11807,0.136157,0.003026,0.016641,0.340393,0.000000,0.000000,0.003026,991.0
3,12103,0.133880,0.581967,0.060109,0.161202,0.000000,0.000000,0.000000,709.0
4,12113,0.038363,0.948849,0.843990,0.000000,0.340153,1.242967,0.913043,2083.0


### 登板試合数

In [10]:
pit_game = train_pitch[['投手ID', '試合ID']].groupby(['投手ID', '試合ID']).count()
pit_game = pd.DataFrame(pit_game.groupby(['投手ID']).size())
pit_game.reset_index(inplace=True)
pit_game.rename(columns={0: 'pit_game_cnt'}, inplace=True)

### イニング数

In [11]:
pit_inning = train_pitch[['投手ID', '試合ID', 'イニング']].groupby(['投手ID', '試合ID', 'イニング']).count()
pit_inning = pd.DataFrame(pit_inning.groupby(['投手ID']).size())
pit_inning.reset_index(inplace=True)
pit_inning.rename(columns={0: 'pit_inning_cnt'}, inplace=True)

### 対戦打者数

In [12]:
pit_batcnt = train_pitch[['投手ID', '試合ID', 'イニング', 'イニング内打席数']].groupby(['投手ID', '試合ID', 'イニング', 'イニング内打席数']).count()
pit_batcnt = pd.DataFrame(pit_batcnt.groupby(['投手ID']).size())
pit_batcnt.reset_index(inplace=True)
pit_batcnt.rename(columns={0: 'pit_batter_cnt'}, inplace=True)

### 投手実績まとめ

In [13]:
pitch_ball = pitch_ball.merge(pit_game, on='投手ID', how='left')
pitch_ball = pitch_ball.merge(pit_inning, on='投手ID', how='left')
pitch_ball = pitch_ball.merge(pit_batcnt, on='投手ID', how='left')
print(pitch_ball.shape)
pitch_ball.head()

(326, 12)


,投手ID,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt
0,11606,0.495751,0.000000,0.000000,0.407932,0.000000,0.000000,0.101983,708.0,46,54,171
1,11766,0.000000,0.967290,0.509346,0.000000,0.051402,0.149533,0.000000,573.0,50,50,154
2,11807,0.136157,0.003026,0.016641,0.340393,0.000000,0.000000,0.003026,991.0,52,61,235
3,12103,0.133880,0.581967,0.060109,0.161202,0.000000,0.000000,0.000000,709.0,24,51,191
4,12113,0.038363,0.948849,0.843990,0.000000,0.340153,1.242967,0.913043,2083.0,23,129,540


In [14]:
pitch_ball.describe()

,投手ID,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt
count,3.260000e+02,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000
mean,1.163288e+06,0.257321,0.461623,0.500294,0.186347,0.138092,0.021658,0.146885,788.702454,20.766871,50.245399,199.684049
std,4.161437e+05,2.295388,0.400976,5.221909,0.227734,0.313514,0.104249,0.633629,756.881660,18.831584,47.533255,192.461884
min,1.160600e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000,1.000000,2.000000
25%,9.253225e+05,0.000000,0.176896,0.000000,0.000000,0.000000,0.000000,0.000000,184.000000,5.250000,11.250000,46.000000
50%,1.300036e+06,0.077167,0.395109,0.000000,0.106137,0.000000,0.000000,0.000000,569.000000,15.000000,37.000000,143.500000
75%,1.500108e+06,0.192714,0.628960,0.233909,0.330539,0.170576,0.000000,0.066470,1031.000000,27.000000,67.750000,263.250000
max,1.700016e+06,41.461538,2.264286,94.153846,1.156364,3.923077,1.242967,10.230769,3004.000000,72.000000,190.000000,756.000000


### 2017年全投手平均
- total: 788.702454
- pit_game_cnt: 20.766871
- pit_inning_cnt: 50.245399
- pit_batter_cnt: 199.684049

In [15]:
pitch_ball.to_feather('intermediate/pit_2017.f')

## 野手
### 打席数

In [16]:
bat_ball = train_pitch[['打者ID', '試合ID', 'イニング', 'イニング内打席数']].groupby(['打者ID', '試合ID', 'イニング', 'イニング内打席数']).count()
bat_ball = pd.DataFrame(bat_ball.groupby(['打者ID']).size())
bat_ball.reset_index(inplace=True)
bat_ball.rename(columns={0: 'batter_cnt'}, inplace=True)

### 試合数

In [17]:
bat_game = train_pitch[['打者ID', '試合ID']].groupby(['打者ID', '試合ID']).count()
bat_game = pd.DataFrame(bat_game.groupby(['打者ID']).size())
bat_game.reset_index(inplace=True)
bat_game.rename(columns={0: 'bat_game_cnt'}, inplace=True)

### 打者成績まとめ

In [18]:
bat_ball = bat_ball.merge(bat_game, on='打者ID', how='left')
print(bat_ball.shape)
bat_ball.head()

(466, 3)


,打者ID,batter_cnt,bat_game_cnt
0,11270,83,35
1,11302,156,69
2,11343,41,28
3,11436,268,81
4,11511,48,21


In [19]:
bat_ball.describe()

,打者ID,batter_cnt,bat_game_cnt
count,4.660000e+02,466.000000,466.000000
mean,1.044992e+06,139.688841,41.448498
std,4.687117e+05,184.754006,45.398623
min,1.127000e+04,1.000000,1.000000
25%,8.000125e+05,7.250000,4.000000
50%,1.100114e+06,44.000000,21.000000
75%,1.400182e+06,222.500000,71.750000
max,1.700019e+06,680.000000,143.000000


### 2017年全野手平均
- batter_cnt: 139.688841
- bat_game_cnt: 41.448498

In [20]:
bat_ball.to_feather('intermediate/bat_2017.f')